# Ridge detection with OpenCV

In [2]:
import os
import cv2
import numpy as np
from osgeo import gdal
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
%matplotlib inline

## read reoreferencing information if availabe

In [1]:
georef = False
filename = 'img/gawler_mag_rtp.tif'
#filename = 'img/Ortho_3_061.png'
outfile = 'Test_ref_cv.tif'

dataset = gdal.Open(filename, gdal.GA_ReadOnly)
if dataset:
    print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                                dataset.GetDriver().LongName))
    print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                        dataset.RasterYSize,
                                        dataset.RasterCount))
    if dataset.GetProjection():
        print("Projection is: {}".format(dataset.GetProjection()))
        geotransform = dataset.GetGeoTransform()
        georef = True
        if geotransform:
            print("Geotransform:" ,geotransform)
            print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
            print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))
            
            gray = np.array(dataset.GetRasterBand(1).ReadAsArray())
    else:
        image = cv2.imread(filename)    
        gray  = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
MASTER = np.zeros(gray.shape, np.double)

NameError: name 'gdal' is not defined

## Operations: ridge detection (sobel) -> thinning -> normalizing

In [2]:
def CVridge(img, ddepth = 'CV_32FC1', dx = 1, dy = 1, ksize = 1, scale = 1, delta = 0, thresh = 0, borderType = 'BORDER_DEFAULT'):
     ridge_filter = cv2.ximgproc.RidgeDetectionFilter_create(
                                                            dx = dx, 
                                                            dy = dy, 
                                                            ksize = ksize, 
                                                            scale = scale, 
                                                            delta = delta
                                                            )
     ridges = ridge_filter.getRidgeFilteredImage(img) 
     n = np.zeros(gray.shape, np.double)
     normalized = cv2.normalize(ridges, n, 1.0, 0.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F) 
     th, dst = cv2.threshold(normalized, thresh, 1, cv2.THRESH_BINARY);    
     return(dst)
  
def InteracRidges(ddepth = 'CV_32FC1', borderType = '', dx = 1, dy = 1, ksize = 1, scale = 1, delta = 0, thresh = 0): 
    f, ax1 = plt.subplots(nrows=1,figsize=(25,25))
    ridges = CVridge(gray, ddepth = ddepth, borderType = borderType, dx = dx, dy = dy, ksize = ksize, 
                     scale = scale, delta = delta, thresh = thresh)
    ax1.imshow(ridges); 
    ax1.get_xaxis().set_visible(False)
    ax1.axes.get_yaxis().set_visible(False)
    x1, x2 = ridges.shape[:2] 
    MASTER[:x1, :x2] = ridges[:x1, :x2]

widgets.interactive(InteracRidges, ddepth = widgets.Dropdown(options=['CV_32FC1','CV_64FC1','CV_16SC1'], value='CV_32FC1'),
                    borderType = widgets.Dropdown(options=['BORDER_CONSTANT','BORDER_REPLICATE','BORDER_REFLECT', 'BORDER_WRAP','BORDER_DEFAULT'], value='BORDER_DEFAULT'),
                    dx = (1,2,1), 
                    dy = (1,2,1), 
                    ksize=(1,7,2), 
                    scale = (0,10,0.01), 
                    delta = (0,1,0.1), 
                    thresh = (0,1,0.01)
                    )

NameError: name 'widgets' is not defined

## write final image to file

In [55]:
if georef:
    driver = gdal.GetDriverByName("GTiff")
    outdata = driver.Create(outfile, dataset.RasterXSize, dataset.RasterYSize, 1, gdal.GDT_Float32)
    outdata.SetGeoTransform(dataset.GetGeoTransform())
    outdata.SetProjection(dataset.GetProjection())
    outdata.GetRasterBand(1).WriteArray(MASTER)
    outdata.FlushCache() 
else:
    cv2.imwrite(outfile, MASTER)
    

In [56]:
outdata = None
band = None
dataset = None